In [196]:
import importlib

In [203]:
import state
import predictor
importlib.reload(predictor)
importlib.reload(state)

<module 'state' from '/home/justiny/Documents/Projects/PyDominion/src/state.py'>

In [198]:
from mlp import PredictorMLP
from mlprunner import train_mlp
from config import GameConfig
from player import load_players
from enums import StartingSplit, FeatureType
from predictor import sample_training_batch, test_mlp
import numpy as np
import torch.nn as nn
import torch
import os
import pandas as pd
from sklearn.metrics import mean_squared_error
from state import ReducedStateFeature
from supply import Supply
from sklearn.preprocessing import StandardScaler

In [265]:
config = GameConfig(split=StartingSplit.StartingRandomSplit, prosperity=False, num_players=2, feature_type=FeatureType.ReducedFeature, sandbox=True)
players = load_players(['R', 'R'], models=None, train=True)

In [323]:
X, y = sample_training_batch(1000, 0.1, config, players)

100%|██████████| 1000/1000 [00:27<00:00, 36.42it/s]


In [267]:
X[0]

array([14.,  0.,  0.,  0.,  8., 17., 26., 25.,  5.,  8.,  4.,  0., 13.,
        4., 21.,  5.,  6.,  4.,  0., 10.,  0.], dtype=float32)

In [301]:
model = PredictorMLP(config.feature_size, (config.feature_size + 1) // 2, 1)
criterion = nn.BCELoss()

In [302]:
project_root = '/home/justiny/Documents/Projects/PyDominion'
model_dir = os.path.join(project_root, 'models')

In [324]:
model_name = 'r-r-mlp-pred-bce-1k-50-1e-1'

In [326]:
y1 = np.array(y, dtype=np.float32)
train_mlp(X, y1, model, criterion, epochs=50, save_epochs=10, model_name=model_name, path=os.path.join(model_dir, model_name))

 10%|█         | 5/50 [01:19<11:56, 15.92s/it]


KeyboardInterrupt: 

In [316]:
y_pred = model(torch.tensor(X).cuda()).detach().cpu().numpy()
y_labels = np.array(y).reshape(y_pred.shape)

In [318]:
output = (y_pred > 0.5)
correct = (output == y_labels).sum() 
acc  = correct / len(y_labels)

In [319]:
acc

0.9999

In [87]:
mean_squared_error(y_pred, y_labels)

0.11117749924175713

Let's try a simpler logistic regression model.

In [163]:
from sklearn.linear_model import LogisticRegression

In [294]:
C = 1
max_iter = 10e5
model = LogisticRegression(max_iter=max_iter, C=C)

In [183]:
project_root = '/home/justiny/Documents/Projects/PyDominion'
model_dir = os.path.join(project_root, 'models')
model_name = 'bm-bm-logistic-pred-1k-10'
model_path = os.path.join(model_dir, model_name)

In [295]:
reg = model.fit(X, y)

In [296]:
reg.score(X, y)

0.9995

Copper, Curse, Estate, Duchy, Province, Silver, Gold

In [299]:
reg.coef_[0][7:14]

array([-0.01494725, -1.96321134,  1.92372774,  5.78549056, 11.25402506,
        0.01330285,  0.01752129])

In [300]:
reg.coef_[0][14:]

array([ 5.97530376e-03,  1.95861603e+00, -1.93016117e+00, -5.95675318e+00,
       -1.12311423e+01, -1.78597002e-02, -2.20421828e-02])

In [283]:
reg.intercept_

array([-1.5037941e-05])

Save the logistic regression model.

In [182]:
import pickle

In [184]:
pickle.dump(reg, open(model_path, 'wb'))

Let's test the logistic regression model.

In [271]:
test_config = GameConfig(split=StartingSplit.StartingRandomSplit, prosperity=False, num_players=2, feature_type=FeatureType.ReducedFeature, sandbox=True)
test_players = load_players(['BM', 'BM'], models=None, train=False)

In [272]:
test_X, test_y = sample_training_batch(1000, -1, test_config, test_players)

100%|██████████| 1000/1000 [00:06<00:00, 145.10it/s]


In [297]:
reg

LogisticRegression(C=1, max_iter=1000000.0)

In [298]:
reg.score(test_X, test_y)

0.999

In [320]:
y_test_pred = model(torch.tensor(test_X).cuda()).detach().cpu().numpy()
y_test_labels = np.array(test_y).reshape(y_test_pred.shape)

In [321]:
output = (y_test_pred > 0.5)
correct = (output == y_test_labels).sum() 
acc  = correct / len(y_test_labels)

In [322]:
acc

0.983